In [73]:
import re
from pathlib import Path
import pandas as pd
import numpy as np
import skfda
from skfda.representation.basis import BSplineBasis
import matplotlib.pyplot as plt
from os import walk

'''
Load point clouds and fit B-spline basis functions 



'''
root_dir = Path("/home/andrew/thesis/prostate_cancer/data/multiseg_proc_output/")
# razer path 
root_dir = Path("/mnt/d/seg+tda+spline___fork1/multiseg_proc_output")

# returns list of dirnames
f = []
for (dirpath, dirnames, filenames) in walk(root_dir):
    f.extend(dirnames)
    break
print(f)

import os
os.scandir(root_dir)

['jrw004_section1_bac2-1.25_crop1_reslice_ bottom', 'jrw004_section2_bac2-1.25_crop1_reslice', 'jrw004_section4_redo_bac2-1.25_crop2_reslice2', 'jrw005_section1_bac2-1.25_crop1_reslice_duplicatebottom-', 'jrw005_section2_bac2-1.25_crop1_reslice_duplicatebottomalso', 'jrw005_section3_bac2-1.25_crop1_reslice', 'jrw005_section4_bac2-1.25_crop1_reslice_dupbottom', 'jrw005_section5_bac2-1.25_crop1_reslice_triplebottom', 'jrw005_section6_bac2-1.25_crop1_reslice_', 'jrw005_section7_bac2-1.25_crop1_reslice_notgreat', 'jrw006_section1_bac2-1.25_crop1_reslice', 'jrw006_section2_bac2-1.25_crop1_reslice', 'JRW006_section3_bac2-1.25_crop1_reslice', 'JRW007_section1_bac2-1.25_crop1_reslice_weirdrepeat', 'JRW007_section2_bac2-1.25_crop1_reslice_alsoweird', 'JRW007_section3_bac2-1.25_crop1_reslice_triplebottom', 'JRW007_section4_bac2-1.25_crop1_reslice', 'JRW007_section5_bac2-1.25_crop1_reslice', 'JRW009_section1_bac2-1.25_crop1_reslice', 'JRW009_section2_bac2-1.25_crop1_reslice', 'JRW015_section1_bac

In [74]:
worklistlist = []
outputdirs = []

# loop through section subfolders and extract absolute path to toporesults into a list
for section in f:
    
    workdir = os.path.join(root_dir, section)

    if os.path.exists(workdir):
        # workdir = os.path.join(workdir, 'topology_results')
        worklist = os.listdir(workdir)
        worklistlist.append(worklist)

        for dirname in walk(workdir):
            print(len(dirname))
            print(dirname)
            # full = os.path.join(workdir, entry)
            #if os.path.isdir(full):
            #    outputdirs.append(full)

    else:
        print(f'The file or directory at {workdir} does not exist.')
    print(workdir)

len(outputdirs)
print(outputdirs)

3
('/mnt/d/seg+tda+spline___fork1/multiseg_proc_output/jrw004_section1_bac2-1.25_crop1_reslice_ bottom', ['segmentation_labels_output'], [])
3
('/mnt/d/seg+tda+spline___fork1/multiseg_proc_output/jrw004_section1_bac2-1.25_crop1_reslice_ bottom/segmentation_labels_output', ['pointclouds', 'topology_results'], ['aggregated_EC.html', 'chunk_metadata.csv', 'full_centroids.csv'])
3
('/mnt/d/seg+tda+spline___fork1/multiseg_proc_output/jrw004_section1_bac2-1.25_crop1_reslice_ bottom/segmentation_labels_output/pointclouds', [], ['chunk_Z0-256_Y0-256_X0-256.csv', 'chunk_Z0-256_Y0-256_X1200-1456.csv', 'chunk_Z0-256_Y0-256_X1440-1696.csv', 'chunk_Z0-256_Y0-256_X1680-1936.csv', 'chunk_Z0-256_Y0-256_X1920-2176.csv', 'chunk_Z0-256_Y0-256_X2160-2416.csv', 'chunk_Z0-256_Y0-256_X240-496.csv', 'chunk_Z0-256_Y0-256_X2400-2633.csv', 'chunk_Z0-256_Y0-256_X480-736.csv', 'chunk_Z0-256_Y0-256_X720-976.csv', 'chunk_Z0-256_Y0-256_X960-1216.csv', 'chunk_Z0-256_Y1200-1456_X0-256.csv', 'chunk_Z0-256_Y1200-1456_X12

In [75]:
def subdirs(root_dir):
    for entry in os.scandir(root_dir):
        yield entry.name

rootscan = subdirs(root_dir=root_dir)
for entry in rootscan:
    print(entry)

jrw004_section1_bac2-1.25_crop1_reslice_ bottom
jrw004_section2_bac2-1.25_crop1_reslice
jrw004_section4_redo_bac2-1.25_crop2_reslice2
jrw005_section1_bac2-1.25_crop1_reslice_duplicatebottom-
jrw005_section2_bac2-1.25_crop1_reslice_duplicatebottomalso
jrw005_section3_bac2-1.25_crop1_reslice
jrw005_section4_bac2-1.25_crop1_reslice_dupbottom
jrw005_section5_bac2-1.25_crop1_reslice_triplebottom
jrw005_section6_bac2-1.25_crop1_reslice_
jrw005_section7_bac2-1.25_crop1_reslice_notgreat
jrw006_section1_bac2-1.25_crop1_reslice
jrw006_section2_bac2-1.25_crop1_reslice
JRW006_section3_bac2-1.25_crop1_reslice
JRW007_section1_bac2-1.25_crop1_reslice_weirdrepeat
JRW007_section2_bac2-1.25_crop1_reslice_alsoweird
JRW007_section3_bac2-1.25_crop1_reslice_triplebottom
JRW007_section4_bac2-1.25_crop1_reslice
JRW007_section5_bac2-1.25_crop1_reslice
JRW009_section1_bac2-1.25_crop1_reslice
JRW009_section2_bac2-1.25_crop1_reslice
JRW015_section1_bac2-1.25_crop1_reslice
JRW015_section2_bac2-1.25_crop1_reslice


In [76]:
""" obj = os.scandir(root_dir)
 
# List all files and directories 
# in the specified path
print("Files and Directories in '% s':" % root_dir)
for entry in obj :
    if entry.is_dir() or entry.is_file():
        print(entry.name) """

def subdirs(path):
    """Yield directory names not starting with '.' under given path."""
    for entry in os.scandir(path):
        if not entry.name.startswith('.') and entry.is_dir():
            yield entry.name

sections = []
with os.scandir(root_dir) as entries:
    for entry in entries:
        # entry.name: file or directory name
        # entry.path: full path to the file or directory
        if entry.is_file():
            print(f"File: {entry.name}")
        elif entry.is_dir():
            print(f"Directory: {entry.name}")
            print(entry.path)
            sections.append(entry.path)

for path in sections:
    os.listdir(path)

Directory: jrw004_section1_bac2-1.25_crop1_reslice_ bottom
/mnt/d/seg+tda+spline___fork1/multiseg_proc_output/jrw004_section1_bac2-1.25_crop1_reslice_ bottom
Directory: jrw004_section2_bac2-1.25_crop1_reslice
/mnt/d/seg+tda+spline___fork1/multiseg_proc_output/jrw004_section2_bac2-1.25_crop1_reslice
Directory: jrw004_section4_redo_bac2-1.25_crop2_reslice2
/mnt/d/seg+tda+spline___fork1/multiseg_proc_output/jrw004_section4_redo_bac2-1.25_crop2_reslice2
Directory: jrw005_section1_bac2-1.25_crop1_reslice_duplicatebottom-
/mnt/d/seg+tda+spline___fork1/multiseg_proc_output/jrw005_section1_bac2-1.25_crop1_reslice_duplicatebottom-
Directory: jrw005_section2_bac2-1.25_crop1_reslice_duplicatebottomalso
/mnt/d/seg+tda+spline___fork1/multiseg_proc_output/jrw005_section2_bac2-1.25_crop1_reslice_duplicatebottomalso
Directory: jrw005_section3_bac2-1.25_crop1_reslice
/mnt/d/seg+tda+spline___fork1/multiseg_proc_output/jrw005_section3_bac2-1.25_crop1_reslice
Directory: jrw005_section4_bac2-1.25_crop1_res

In [94]:
import os
# linux path
root_dir = Path("/home/andrew/thesis/prostate_cancer/data/multiseg_proc_output/")
# razer path 
root_dir = Path("/mnt/d/seg+tda+spline___fork1/multiseg_proc_output")

# returns list of dirnames
section = []
for (dirpath, dirnames, filenames) in walk(root_dir):
    section.extend(dirnames)
    break
print(section)

segout = []
for root, dirs, files in os.walk(root_dir):
    for d in dirs:
        if 'output' in d:
            abs_path = os.path.abspath(os.path.join(root, d))
            segout.append(abs_path)
#
pcpath = []
for a in segout:
    for root, dirs, files, in os.walk(a):
        for b in dirs:
            if 'point' in b:
                pc_add = os.path.abspath(os.path.join(root, b))
                pcpath.append(pc_add)

filtpath = []
for a in segout:
    for root, dirs, files, in os.walk(a):
        for b in dirs:
            if 'topology' in b:
                filtpath_add = os.path.abspath(os.path.join(root, b))
                filtpath.append(filtpath_add)

# print(len(segout))
# print(len(filtpath))
#
print(segout)
print(pcpath)
print(filtpath)

for a in filtpath:
    a = Path(a)
    print(a.parts)

['jrw004_section1_bac2-1.25_crop1_reslice_ bottom', 'jrw004_section2_bac2-1.25_crop1_reslice', 'jrw004_section4_redo_bac2-1.25_crop2_reslice2', 'jrw005_section1_bac2-1.25_crop1_reslice_duplicatebottom-', 'jrw005_section2_bac2-1.25_crop1_reslice_duplicatebottomalso', 'jrw005_section3_bac2-1.25_crop1_reslice', 'jrw005_section4_bac2-1.25_crop1_reslice_dupbottom', 'jrw005_section5_bac2-1.25_crop1_reslice_triplebottom', 'jrw005_section6_bac2-1.25_crop1_reslice_', 'jrw005_section7_bac2-1.25_crop1_reslice_notgreat', 'jrw006_section1_bac2-1.25_crop1_reslice', 'jrw006_section2_bac2-1.25_crop1_reslice', 'JRW006_section3_bac2-1.25_crop1_reslice', 'JRW007_section1_bac2-1.25_crop1_reslice_weirdrepeat', 'JRW007_section2_bac2-1.25_crop1_reslice_alsoweird', 'JRW007_section3_bac2-1.25_crop1_reslice_triplebottom', 'JRW007_section4_bac2-1.25_crop1_reslice', 'JRW007_section5_bac2-1.25_crop1_reslice', 'JRW009_section1_bac2-1.25_crop1_reslice', 'JRW009_section2_bac2-1.25_crop1_reslice', 'JRW015_section1_bac

In [95]:
from collections import defaultdict

# Create a nested dictionary where the first level keys are sample names
# and the second level keys are section names.
data = defaultdict(dict)

# Suppose you've parsed your absolute path and extracted the following:
sample = "jrw004"
section = "section1"
topology_results_path = (
    "/mnt/d/seg+tda+spline___fork1/multiseg_proc_output/"
    "jrw004_section1_bac2-1.25_crop1_reslice_bottom/segmentation_labels_output/topology_results"
)

# You can store details about the topology results.
# This could include file paths, loaded CSV data (e.g., as pandas DataFrames), etc.
data[sample][section] = {
    "topology_results_path": topology_results_path,
    # Add more details here as needed
}

# Accessing the data:
print("Topology results for", sample, "in", section, "are located at:")
print(data[sample][section]["topology_results_path"])


Topology results for jrw004 in section1 are located at:
/mnt/d/seg+tda+spline___fork1/multiseg_proc_output/jrw004_section1_bac2-1.25_crop1_reslice_bottom/segmentation_labels_output/topology_results


In [78]:
import os 
from os.path import join, getsize

for root, dirs, files, in os.walk(root_dir):
    

SyntaxError: incomplete input (4009845921.py, line 5)

In [ ]:
from glob import glob
w = [] # list of sub subfolder dirpaths
ww = [] # list of subfolder output dir names
for path in sections:
    for dirpath in walk(path):
        for dirnames in walk(dirpath):
            a = glob.glob(os.path.join(dirname, '*output*'))
            print(len(a))
        #print(dirpath)
        w.append(dirpath)
    #print(os.listdir(path))

len(w)

TypeError: expected str, bytes or os.PathLike object, not tuple